In [18]:
!pip install flask pyngrok

In [19]:
from flask import Flask, request, redirect, render_template_string
from pyngrok import ngrok
import sqlite3, string, random

In [20]:
ngrok.set_auth_token("35WepmSHt4R6D65b6kC5HcXZy76_3DX17a6axgFHCxBNVuLeQ")

In [21]:
conn = sqlite3.connect('url_shortener.db', check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS urls (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    short_code TEXT UNIQUE,
    original_url TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    access_count INTEGER DEFAULT 0
)
''')
conn.commit()

print("Database initialized successfully!")



Database initialized successfully!


In [22]:
def generate_short_code(length=6):
    chars = string.ascii_letters + string.digits
    return ''.join(random.choices(chars, k=length))


In [23]:
app = Flask(__name__)

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>Parsh’s URL Shortener</title>
    <style>
        body { font-family: Arial; padding: 20px; background: #f2f2f2; }
        .box { background: white; padding: 20px; border-radius: 10px; width: 400px; margin: auto; }
        input, button { width: 100%; padding: 10px; margin-top: 10px; }
        button { background: #007bff; color: white; border: none; border-radius: 5px; }
        .output { margin-top: 20px; padding: 10px; background: #e6ffe6; border-left: 5px solid green; }
    </style>
</head>
<body>
    <div class="box">
        <h2>URL Shortener</h2>
        <form method="POST">
            <input name="url" placeholder="Enter long URL">
            <button type="submit">Shorten URL</button>
        </form>

        {% if short_url %}
        <div class="output">
            <p><b>Short URL:</b> <a href="{{ short_url }}" target="_blank">{{ short_url }}</a></p>
        </div>
        {% endif %}
    </div>
</body>
</html>
'''


In [24]:
@app.route("/", methods=["GET", "POST"])
def home():
    short_url = None

    if request.method == "POST":
        long_url = request.form["url"]
        short_code = generate_short_code()

        cursor.execute(
            "INSERT INTO urls (short_code, original_url) VALUES (?, ?)",
            (short_code, long_url)
        )
        conn.commit()

        short_url = public_url + "/" + short_code

    return render_template_string(HTML_TEMPLATE, short_url=short_url)

In [25]:
@app.route("/<short_code>")
def redirect_url(short_code):
    cursor.execute(
        "SELECT original_url FROM urls WHERE short_code = ?",
        (short_code,)
    )
    result = cursor.fetchone()

    if result:
        return redirect(result[0])
    else:
        return "Invalid short URL!", 404

In [26]:
public_url = ngrok.connect(5000)
print("🔗 Your Live URL Shortener:", public_url)

app.run(port=5000)

🔗 Your Live URL Shortener: NgrokTunnel: "https://unhaggled-ching-undegenerate.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [27]:
from google.colab import files
files.download('url_shortener.db')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>